Social Data Science WS19/20

# Home Assignment 2


Submit your solution via Moodle until 23.59pm on Wednesday, November 27th. Late submissions are accepted for 12 hours following the deadline, with 1/3 of the total possible points deducted from the score.

Submit your solutions in teams of up 2-3 members. **Single student submissions will not be graded anymore.**
Please denote all members of the team with their student id and full name in the notebook. In this home assignment, you have to submit an .ipynb notebook and a file "beta.npy" as specified in task 3. Do not submit anything else than these two files!

Cite ALL your sources for coding this home assignment. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members will be expelled from the course without warning.

If you have any general questions regarding this assignment, please ask on Moodle.

##### List team members, including all student IDs here:
1. Romayssa
2. Moncef=


In [22]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from scipy.spatial import distance
import matplotlib.pyplot as plt
import seaborn as sns

### Task 1: Causal Inference (1 pt)

Suppose you conduct a study to evaluate the effect of a new procedure for coronary bypass surgery that is supposed to help with the postoperative healing process. Since the new procedure is more risky than the old one, it is rarely performed on patients who are over 80 years old. However, there is also hardly any data on under 80-year olds that have taken the old treatment. 

You can find the data from the study in "bypass.csv", where _stay_ is the length of the hospital stay after surgery, _age_ is the age of the patient, and _new_ is the binary indicator variable specifying whether the new surgical procedure was used. Additionally, there is a column _severity_ which quantifies the severity of preoperative diseases.

Perform a regression analysis to draw a causal inference whether the new surgery method shortens the length of the postoperative hospital stay. Can you observe a significant effect? Would you generally recommend to apply the new surgery method?
Provide a thorough explanation of your answers and the steps in your analysis!

#  <center> **Steps of the analysis** </center>
*The goal is to evaluate the effect of the new surgery. For this we want to prove that the difference of the stay period for an individual under the new surgery method is really different from the stay period under the old surgery method.
As a matter of fact we want to proove that the new surgery method is in CAUSE (inference of causality) of the changement of the stay period.*

</br>
As mentionned there are few problems with our Data:

1.   More individulas with the new surgery method than the old one (168 vs 32 enteries).
2.   Age is unbalanced: much more individuals with new surgery method are younger than 80 and much more individuals are older than 80 with the old surgery.
3. An individual is also characterized by his age and also the severity of his health. 

These problems make the study of correlation between the **surgery method** and the **stay** period and the causality inference ambiguous and not effective.
</br>

</br>

### Step 1: Matching Data
To tackle these probelms we will use a method seen in the 

lecture, the **Matching Method**:

- We will devide our data into two groups: old method data points and new method data points. 

- We will apply a matching method by forming two groups that are similar by age and also severity and that had new method or old method surgery.

- We will generate a new Dataset with this matched Data that can be analyzed.


### Step 2: Hypothesis test

- null hypothesis:


    There is a no difference in stay between the old surgery method and the new surgery method.
- alternative hypothesis:


    The new surgery method decrease significatly the stay period.

</br>

#### How to perform this test hypothesis ?

- With this new Dataset, we will compare the mean of the stay period for the two groups (new method and old method).
- We will also do a multi-attribute regression inference.to see if the surgery method correlate with the stay. 

In [2]:
#load the file
data= pd.read_csv("bypass.csv") #load file

In [8]:
## function from the instruction of Social DS. that do a greedy matching between two populations
# We assume that group 1 is bigger than group 2
def greedy_matching (group1,group2):
    group_1 = group1.copy()
    
    # get inverse covariance matrix for mahalanobis distance
    cov = np.cov(np.vstack([group1, group2]).T) # note that numpy's cov() function computes covariances between ROWS
    invcov = np.linalg.inv(cov)
    # for each element in the smaller group pick one greedily from the first group
    matching = {}
    for i,o2 in enumerate(group2):
        #compute distances:
        distances = [distance.mahalanobis(o2, o1, invcov) for o1 in group1]
        #find the index of the best match for element o2
        best_match = np.argmin(distances)
        #store the matching
        matching [i] = best_match
        #prevent matching the same best_match again
        group_1[best_match].fill(np.inf)     
    return matching

# specify columns to match on and both new and old surgery groups
match_cols = ['age','severity']

old_group = data.loc[data.new == 0, match_cols]
new_group = data.loc[data.new == 1, match_cols]
matching = greedy_matching(old_group.to_numpy(), new_group.to_numpy())
matching

# get resulting control group after matching
all_cols = ['age','severity','new','stay']
matched_old_group = data.loc[data.new==0,all_cols].reset_index(drop=True)
matched_old_group = matched_old_group.iloc[list(matching.values())].reset_index(drop=True)
matched_old_group
matched_new_group = data.loc[data.new==1,all_cols].reset_index(drop=True)
matched_new_group
# stack matched data
matched_data = matched_new_group.append(matched_old_group)
matched_data.info()
matched_data.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 167
Data columns (total 4 columns):
age         336 non-null float64
severity    336 non-null float64
new         336 non-null int64
stay        336 non-null float64
dtypes: float64(3), int64(1)
memory usage: 13.1 KB


,age,severity,new,stay
count,336.000000,336.000000,336.000000,336.000000
mean,73.151058,49.382448,0.500000,27.779566
std,7.246904,9.128871,0.500746,4.465878
min,50.637442,26.687629,0.000000,15.357648
25%,68.977574,41.164295,0.000000,24.376341
50%,76.224462,47.904978,0.500000,28.447573
75%,78.797649,56.915571,1.000000,30.881043
max,83.580951,72.054950,1.000000,36.750665


In [6]:
mean_new = np.mean(matched_data.loc[matched_data.new == 1,"stay"])
mean_old = np.mean(matched_data.loc[matched_data.new == 0,"stay"])
print("The mean of stay with the new surgery method:",mean_new)
print("The mean of stay with the old surgery method:",mean_old)

The mean of stay with the new surgery method: 24.660080097356843
The mean of stay with the old surgery method: 30.899051922951518


In [7]:
#We perform an Ordinary Least Squares Regression

pred_cols = ['age','new','severity']
X = matched_data[pred_cols]
X= sm.add_constant(matched_data[pred_cols])
y = matched_data.stay
model = sm.OLS(y, X).fit()

model.summary()

C:\Users\Joe\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   stay   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 3.051e+04
Date:                Wed, 27 Nov 2019   Prob (F-statistic):               0.00
Time:                        20:27:55   Log-Likelihood:                -34.426
No. Observations:                 336   AIC:                             76.85
Df Residuals:                     332   BIC:                             92.12
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7795      0.220      3.547      0.000       0.347       1.212
age            0.2010      0.003     68.510      0.000       0.195       0.207
new           -4.6257      0.042   -110.309      0.000      -4.708      -4.543
severity       0.2959      0.002    174.237      0.000       0.293       0.299
==============================================================================
Omnibus:                        0.397   Durbin-Watson:                   2.234
Prob(Omnibus):                  0.820   Jarque-Bera (JB):                0.525
Skew:                          -0.049   Prob(JB):                        0.769
Kurtosis:                       2.833   Cond. No.                     1.34e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.34e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Results 

We notice the difference in the means of period of stay between the two sugery methods is significant 24.6 against 30.89.

The linear regression shows that the age and the sevirity do influence the period of healing but not as much as the new surgery method. The surgery method is highly correlated to the stay attribute ~ - 4.62. As our data is a matched data we can infere a causality relation between the new surgery and the stay period. 

Back to our hypothesis: We notice that the T-value is -110.309 and the P-value is null. (We have a negative influence). We refute the null hypothesis and accept the alternative: The new surgery method reduce the period of stay in the hospital.

**As a result: We recommand the use of this new surgery method.**


### Task 2: Naive Bayes on Continuous Data (1 pt)

In lecture we have introduced the _Naive Bayes_ classifier, in which for each feature, we compute the empirical probabilities 
$ P(x_i|y) $ to form our predictions. For discrete data, we have learned in lecture that these probabilities are estimated from the raw class-conditional feature counts. For continuous features however, this approach obviously does not make sense.
Instead, for continuous features, one usually makes the assumption that these  are normally distributed. Thus for continuous features, one usually models the class-conditional probabilities via 
$$ P(X_i = x| y = C_k) = \frac{1}{\sqrt{2\pi \sigma_{ik}^2}} \exp\left(-\frac{(x_i-\mu_{ik})^2}{2\sigma_{ik}^2}\right)$$
where $\mu_{ik}$ and $\sigma_{ik}$ are the mean and variance of the features $X_i$ that are classed as $c_k$.
A Naive Bayes classifier for which all features are assumed to be continuous and normally distribited is also called _Gaussian Naive Bayes_ classifier.

Implement a class that performs Naive Bayes classification on both discrete and continuous data, using the class and function signatures and specifications in the cell below, which are designed to be similar to the model classes in sklearn. 
When passing your data to the fit() function, by default your classifier should treat all columns as continuous, i.e. normally distributed as explained above. Discrete features have to be specified upon fitting, by passing a list of integers specifying the indices of the columns that are to be treated as discrete features.

Note: You may use any function from the numpy library, but __NO__ function from the scikit-learn library. Further, you may assume that all input data is valid, i.e. you do not need to check whether for instance the feature matrix X has as many rows as the class vector $y$.

In [ ]:
import numpy as np
import pandas as pd
class NaiveBayes:
    def __init__(self):
        self.dataset_classes = []
        self.discrete_columns = []
        self.probabilities = {}

    # fit your model, nothing is returned
    # X: two dimensional numpy array describing the feature matrix to train on
    # y: one dimensional numpy array or list representing the class vector used in training
    # discrete columns: optional list of indices describing the coumns in X that are to be treated as numerical.
    def fit(self, X, y, discrete_columns=None):
        self.dataset_classes = list(set(y))
        self.discrete_columns = discrete_columns
        X = pd.DataFrame(data=X, columns=None)
        X["class_"] = y
        for i, col in enumerate(X.columns[:-1]):
            if discrete_columns and i in discrete_columns:
                # Processing a discrete column
                for val in set(X.iloc[:, i]):
                    for class_ in self.dataset_classes:
                        self.probabilities[f"{val}_{class_}"] = X[(X[col] == val) & (X["class_"] == class_)].shape[0] / \
                                                                X[(X[col] == val)].shape[0]
            else:
                # Processing continuous column
                for class_ in self.dataset_classes:
                    target = X[X["class_"] == class_][col]
                    variance = np.var(target)
                    mean = np.mean(target)
                    self.probabilities[f"{col}_{class_}"] = {'variance': variance, 'mean': mean}
        for class_ in self.dataset_classes:
            self.probabilities[class_] = X[(X["class_"] == class_)].shape[0] / X["class_"].shape[0]

    def __proba_calc(self, row, class_):
        proba = 1
        for i, val in enumerate(row):
            if self.discrete_columns and i in self.discrete_columns:
                # Processing a discrete column
                key = f"{val}_{class_}"
                proba *= self.probabilities[key]
            else:
                # Processing continuous column
                key = f"{i}_{class_}"
                proba *= (1 / np.sqrt(2 * np.pi * self.probabilities[key]["variance"])) * np.exp(
                    (-1) * np.power(val - self.probabilities[key]["mean"], 2) / (
                                2 * self.probabilities[key]["variance"]))
        return proba

    # predict function: use the previously trained model to make a prediction
    # -> return a vector y_hat of predicted classes.
    # X: two dimensional numpy array describing the feature matrix to predict on
    def predict(self, X):
        y_hat = []
        for row in X:
            proba = []
            for class_ in self.dataset_classes:
                proba.append(self.__proba_calc(row, class_)*self.probabilities[class_])
            y_hat.append(self.dataset_classes[np.argmax(proba)])
        return y_hat

### Task 3: Predicting House Prizes (3 pts)

In this task you have to create a regression model to predict housing prices. We are providing you with a training dataset that you have to use to form your model, but we are going to grade your model based on the performance on a test set that is not known to you.

__Grading:__ We will measure the performance of your model in terms of the MSE error on a test set that we are holding out. We will create a simple baseline model ourselves, which will be based on a little feature engineering, and a ridge regression model that is just run with default parameters from scikit-learn. To obtain 50% of the points in this task, your model should show a better performance on the test set than our baseline. The remaining 50% of the points will be awarded based on your performance against the rest of the class. The best models will get full marks, and the rest of the marks will degrade along with the testing error scores.


__What to submit:__ In your submission, you have to provide the following things: 
1. Two functions that preprocess the input data into the format that you chose to use in your final regression model. The first function should perform the main part of the feature engineering and return an unscaled predictor matrix. Thus, its input has to be a pandas dataframe consisting of all the predictor colums in the data, and its output has to be a two-dimensional numpy array $X$. The second function should scale the preprocessed matrix. Its input is a training matrix that your scaling model is based on, and a test matrix that your scaling model is then to be applied on, to return a scaled test matrix. The exact signatures and the testing pipeline can be found below. Note that the scaling function may also perform other things than scaling, e.g. adding a constant column or adding interactions, as long as the testing pipeline below works properly. 
2. A corresponding parameter vector $\beta$, that is saved as a numpy array into the file "beta.npy". It is used to compute the model predictions via $\hat{y} = X\beta$. Note that any other models next to regression models are not admissible.
3. All the code that you used to optimize your final model. Submissions that show no code in that regard will not be awarded any points. Further, we encourage to report all the exploratory analysis that you did, and report some of the errors that your model achieved on some validation data. This may grant some extra points in the case that your model does not outperform our baseline on the testing data.

Note that in this task, you may use any Python library that can be installed via pip. In particular, you may use libraries like statsmodel or scikit-learn. 

In the cells below, you can find the signatures of the preprocessing and scaling functions, the pipeline that we are going to use to make predictions, and some space where your code for exploration and optimization can go. Note that your model, i.e. both preprocessing functions and the parameter vector have to work on this pipeline. If this pipeline cell does not execute, you will not get any points. 

In [233]:
# signature of preprocessing & scaling functions - you MUST use these function signatures in your final submission.

def preprocess(dfX):
    # insert your code here
    dfx=dfX
    dfx= pd.get_dummies(dfx, prefix='HeatingFuel_', columns=['HeatingFuel'])
    dfx= pd.get_dummies(dfx, prefix='HeatingSystem_', columns=['HeatingSystem'])
    dfx= pd.get_dummies(dfx, prefix='SewerSystem_', columns=['SewerSystem'])
    dfx["HasCentralAC"] = list(map(lambda x: 1 if x.lower() == 'yes' else 0,dfx["HasCentralAC"]))
    dfx["AtWaterfront"] = list(map(lambda x: 1 if x.lower() == 'yes' else 0,dfx["AtWaterfront"]))
    columns=["LivingArea","LandValue","PropertySize","Age","NeighborhoodHigherEdPct","Bathrooms","Rooms","Fireplaces","HasCentralAC"]
    dfx=dfx[columns]
    X=dfx.values
    return X #X has to be a two-dimensional numpy array

# inputs are both numpy matrices
def scale(X_train, X_test):
    # insert your code here
    mm_scaler = preprocessing.MaxAbsScaler()
    mm_scaler.fit(X_train)
    X_test_scaled=mm_scaler.transform(X_test)
    return X_test_scaled  #X has to be a two-dimensional numpy array as well

In [234]:
# testing pipeline, for illustration. Test data will have the same format as the training data. Do NOT edit this cell, 
# but feel free to copy it to test it yourself.  

from sklearn.metrics import mean_squared_error

# read in data
df_train = pd.read_csv("housing_train.csv")
df_test = pd.read_csv("housing_test.csv")

# split predictor dataframe from complete data
dfX_train = df_train.iloc[:,:-1]
dfX_test = df_test.iloc[:,:-1]

# preprocess training and test data - preprocessed training data is always needed for scaling
X_train = preprocess(dfX_train)
X_test = preprocess(dfX_test)

# finally, scale your data into a proper format. Note that for scaling the training data,
# you should call 'X_train_scaled = scale(X_train, X_train)'
X_test_scaled = scale(X_train,X_test)

print(X_test_scaled)

# load parameter vector resulting from your optimized model
beta = np.load("beta.npy")

# apply your vector to predict on the test data
y_pred = np.dot(X_test_scaled,beta)


# get target column from test data and compute MSE
y_test = df_test.iloc[:,-1].to_numpy()
print(mean_squared_error(y_test,y_pred))

[[0.67627494 0.42705731 0.06491689 ... 1.         0.25       0.        ]
 [0.18847007 0.01529411 0.07181132 ... 0.33333333 0.         0.        ]
 [0.26607539 0.07686308 0.0193522  ... 0.5        0.25       0.        ]
 ...
 [0.36807095 0.13921715 0.0455647  ... 0.58333333 0.25       1.        ]
 [0.4345898  0.05098035 0.05385849 ... 0.83333333 0.25       0.        ]
 [0.2172949  0.08784424 0.02208266 ... 0.58333333 0.25       0.        ]]
26638134831.717613


You can illustrate your process of model optimization in the cell(s) below. Feel free to add cells where necessary.

In [224]:

#Step 1: Data processing
#Everything dummies
data=pd.read_csv("housing_train.csv")
data= pd.get_dummies(data, prefix='HeatingFuel_', columns=['HeatingFuel'])
data= pd.get_dummies(data, prefix='HeatingSystem_', columns=['HeatingSystem'])
data= pd.get_dummies(data, prefix='SewerSystem_', columns=['SewerSystem'])
data["HasCentralAC"] = list(map(lambda x: 1 if x.lower() == 'yes' else 0,data["HasCentralAC"]))
data["AtWaterfront"] = list(map(lambda x: 1 if x.lower() == 'yes' else 0,data["AtWaterfront"]))

#Delete outliers -- need to see this
     #We do two verison with and without outliers and we compare the best  
"""
data = data[data["PropertySize"]<4150]
data = data[data["Age"]<53]
data = data[data["LandValue"]<41000]

data = data[data["LivingArea"]<270]
data = data[data.Bedrooms.between(2,5)]
data = data[data.Price.between(25000,250000)]
"""

#Data Correlation
matrix = data.corr()
interesting_variables = matrix['Price'].sort_values(ascending=False)
#print(interesting_variables)

#Step 3 Features selection
#We did also a Random Forest Regressor to show the most significant features
columns=["LivingArea","LandValue","PropertySize","Age","NeighborhoodHigherEdPct","Bathrooms","Rooms","Fireplaces","HasCentralAC","Price"]
data=data[columns]
print(data.shape)

(1200, 10)


In [208]:
#Here we will compare different models
import numpy as np
from sklearn.linear_model import LinearRegression,Lasso, RidgeCV
from sklearn.model_selection import train_test_split 
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn import preprocessing

array = data.values
y=data["Price"]

####deleting the target
del(data['Price'])
##### the data to learn in a vector --x
x=data.values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=2)
X_test=scale(X_train, X_test)

#We did an alpha testing and found out that our best alpha =0.02
#We found out that Lasso is the best Regression that minimizes the Data
reg = Lasso(alpha=0.02)
reg.fit(X_train,y_train)
reg.score(X_test, y_test)

np.save('beta',reg.coef_)

y_pred = np.dot(X_test,reg.coef_)
print(mean_squared_error(y_test,y_pred))


26825164540.088383
